<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2005%20-%20LlamaIndex_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Load the autoreload extension
%load_ext autoreload

# Set autoreload to mode 2 to reload all modules automatically
%autoreload 2

In [2]:
%pip install -q langchain openai python-dotenv pypdf deeplake langchain-community llama-index-vector-stores-deeplake

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

# os.environ['ACTIVELOOP_TOKEN']

# Set the API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

openai.api_key = api_key

In [22]:
import logging
import sys

#You can set the logging level to DEBUG for more verbose output,
# or use level=logging.INFO for less detailed information.
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# LlamaHub Wikipedia Integration

In [23]:
from llama_index.core.readers.download import download_loader

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

/tmp/ipykernel_48808/167685543.py:3: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")


In [24]:
documents = loader.load_data(pages=['Nicolas Maduro', 'Maria Corina Machado'])

In [5]:
len( documents )

2

In [6]:
documents[2]

Document(id_='43446146', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Benjamín Rausseo Rodríguez (born January 26, 1961) is a Venezuelan artist, stand-up comedian and career humorist best known for his character El Conde del Guácharo (translation: "The Count of Guacharo").\n\n\n== Biography ==\nBenjamín Rausseo was born into poverty in the rural community of Musipán in the state of Monagas. \nHe pursued university studies as an actor and, in 1981, presented as his graduation thesis a comedic monologue "el Conde del Guácharo." The "Conde" or Count became a TV personality and made Rausseo one of Venezuela\'s most successful stand-up comedians.  Beyond comedy he was also a dare-devil and stunt performer in the television variety show Sabado Sensacional. In 2006 Rausseo was studying law and was one semester from graduating with a JD from the Universidad Santa María de Caracas. Rausseo owns a theme park in Isla Margarit

In [25]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

def embedding_function(texts, model="text-embedding-3-small"):
    if isinstance(texts, str):
        texts = [texts]
    texts = [t.replace("\n", " ") for t in texts]
    try:
        response = client.embeddings.create(input=texts, model=model)
        return [data.embedding for data in response.data]
    except Exception as e:
        print(f"Error in embedding function: {str(e)}")
        return None

# Save on DeepLake

In [8]:
pip install llama-index-vector-stores-deeplake

Note: you may need to restart the kernel to use updated packages.


In [9]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

DEBUG:PIL.Image:Importing BlpImagePlugin
Importing BlpImagePlugin
DEBUG:PIL.Image:Importing BmpImagePlugin
Importing BmpImagePlugin
DEBUG:PIL.Image:Importing BufrStubImagePlugin
Importing BufrStubImagePlugin
DEBUG:PIL.Image:Importing CurImagePlugin
Importing CurImagePlugin
DEBUG:PIL.Image:Importing DcxImagePlugin
Importing DcxImagePlugin
DEBUG:PIL.Image:Importing DdsImagePlugin
Importing DdsImagePlugin
DEBUG:PIL.Image:Importing EpsImagePlugin
Importing EpsImagePlugin
DEBUG:PIL.Image:Importing FitsImagePlugin
Importing FitsImagePlugin
DEBUG:PIL.Image:Importing FliImagePlugin
Importing FliImagePlugin
DEBUG:PIL.Image:Importing FpxImagePlugin
Importing FpxImagePlugin
DEBUG:PIL.Image:Image: failed to import FpxImagePlugin: No module named 'olefile'
Image: failed to import FpxImagePlugin: No module named 'olefile'
DEBUG:PIL.Image:Importing FtexImagePlugin
Importing FtexImagePlugin
DEBUG:PIL.Image:Importing GbrImagePlugin
Importing GbrImagePlugin
DEBUG:PIL.Image:Importing GifImagePlugin
Impor

/teamspace/studios/this_studio/.conda/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.18) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [26]:
import deeplake
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

my_activeloop_org_id = "macayaven"
my_activeloop_dataset_name = "LlamaIndex_intro"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

try:
    # Create the vector store
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=True,
        runtime={"tensor_db": True},
        embedding_function=embedding_function
    )
    print("Vector store created successfully.")

    # Create the storage context
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create the index
    index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context
    )
    print("Index created successfully.")

except Exception as e:
    print(f"An error occurred: {str(e)}")

Your Deep Lake dataset has been successfully created!


Vector store created successfully.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Uploading data to deeplake dataset.


100%|██████████| 25/25 [00:00<00:00, 89.60it/s]
|

Dataset(path='hub://macayaven/LlamaIndex_intro', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (25, 1)      str     None   
 metadata     json      (25, 1)      str     None   
 embedding  embedding  (25, 1536)  float32   None   
    id        text      (25, 1)      str     None   
Index created successfully.


In [27]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [28]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
print('hola')

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Uploading data to deeplake dataset.


100%|██████████| 25/25 [00:00<00:00, 362.12it/s]
\

Dataset(path='hub://macayaven/LlamaIndex_intro', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (50, 1)      str     None   
 metadata     json      (50, 1)      str     None   
 embedding  embedding  (50, 1536)  float32   None   
    id        text      (50, 1)      str     None   
hola


# Retrieve to answer questions

In [29]:
query_engine = index.as_query_engine()

In [30]:
response = query_engine.query("Who is more likely to win a marathon, Maria Corina Machado or Nicolás Maduro. Justify your answer explaining the rationale of your answer.")
print(response.response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Maria Corina Machado is more likely to win a marathon than Nicolás Maduro. This conclusion is based on the information provided in the context, which highlights Machado as an energetic former legislator who has been actively involved in political campaigns, marches, and leading the opposition movement against the current government in Venezuela. On the other hand, Nicolás Maduro is the current president of Venezuela and is not portrayed as actively engaging in physical activities or public events that would suggest he has the stamina and physical fitness required to win a marathon.


In [16]:
questions = [
    "What are the main policy differences between the two candidates?",
    "How do international relations differ for each candidate?",
    "What economic policies does each candidate propose?"
]

for question in questions:
    response = query_engine.query(question)
    print(f"Question: {question}")
    print(f"Answer: {response.response}\n")

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7f1f77ab0ca0>, 'json_data': {'input': ['What are the main policy differences between the two candidates?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x7f1f77ab0ca0>, 'json_data': {'input': ['What are the main policy differences between the two candidates?'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:openai._base_client:Sending HTTP Request: POST https://api.openai.com/v1/embeddings
Sending HTTP Request: POST https://api.openai.com/v1/embeddings
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address

## Let's start fresh and use the stored embeddings

In [8]:
%pip install --upgrade --force llama-index llama-index-vector-stores-deeplake openai

  Using cached llama_index-0.10.62-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_vector_stores_deeplake-0.1.6-py3-none-any.whl.metadata (709 bytes)
  Using cached openai-1.40.1-py3-none-any.whl.metadata (22 kB)
  Using cached llama_index_agent_openai-0.2.9-py3-none-any.whl.metadata (729 bytes)
  Using cached llama_index_cli-0.1.13-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.10.62-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_openai-0.1.11-py3-none-any.whl.metadata (655 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.2.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_legacy-0.9.48-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.1.29-py3-none-any.whl.metadata (650 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.1.9-py3-none-any.whl.metadata (728 bytes)
  Using cached llama_index_program_openai-0.1.7-py3-none-any.whl.metadata (760 bytes)
  Using cached 

In [31]:
import os
from llama_index.core import VectorStoreIndex, StorageContext, Settings
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding

# Create an instance of the OpenAI embedding model
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Set the embedding model as the default
Settings.embed_model = embed_model

In [32]:
my_activeloop_org_id = "macayaven"
my_activeloop_dataset_name = "LlamaIndex_intro"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create the vector store
vector_store = DeepLakeVectorStore(
    dataset_path=dataset_path,
    read_only=True
)

Deep Lake Dataset in hub://macayaven/LlamaIndex_intro already exists, loading from the storage


In [33]:
print(f"Number of items in vector store: {vector_store}")

Number of items in vector store: stores_text=True is_embedding_query=True flat_metadata=True ingestion_batch_size=1024 num_workers=4 token=None read_only=True dataset_path='hub://macayaven/LlamaIndex_intro'


In [34]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load the index from the existing vector store
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context
)

In [35]:
# Create a query engine
query_engine = index.as_query_engine()

# Ask a question
question = "Who of the two candidates would be a better president for Venezuela and why? Analyze it in bullet points. And end with a summary. The goal is to make Venezuela a great country again."
response = query_engine.query(question)

# Print the response
print(response.response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
- Maduro was chosen by Chávez as his potential successor and received his endorsement, which could indicate a continuation of Chávez's policies and vision for Venezuela.
- Maduro narrowly won the presidential election in 2013, which may suggest some level of popular support for his leadership.
- Maduro's government has faced criticism for alleged delays and obstacles in the recall referendum process, which could raise concerns about democratic practices under his leadership.
- Maduro's proposal for the 2017 Venezuelan Constituent Assembly election, despite international condemnation, could indicate a willingness to push through controversial decisions for what he believes is in the country's best interest.

Summary: Maduro's close ties to 

# Environment

In [19]:
!pip list > '/teamspace/studios/this_studio/ragbook-notebooks/requirements/Chapter 05 - LlamaIndex_Introduction/requirements.txt'


5681.12s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
